In [195]:
import numpy as np
import matplotlib as plt
from math import *

In [196]:
class Plaintext:
  """Class with different computations for a plaintext object
  """
  def __init__(self, pslots):
    """
    slots: 1D array
    """

    self.slots = np.array(pslots)
    self.num_slots = len(pslots)

  def __str__(self):
    return str(self.slots)

  def add(self, other_plain):
    """Method computes the sum of two plaintext objects
  Returns:
    Plaintext sum of self and other_plain
  """

    result = self.slots + other_plain.slots
    result1 = Plaintext(result)
    return result1

  def mul(self, other_plain):
    """Method computes the multiplication of two plaintext objects
  Returns:
    Plaintext product of self and other_plain
  """

    multresult = self.slots * other_plain.slots
    multresult1 = Plaintext(multresult)
    return multresult1

  def rotate(self, index):
    """Method computes the rotation of an object for a specific index
  Args:
    index: number of slots to rotate by
  Returns:
    Rotated plaintext object
    ex: rotating [1,2,3,4] by 1 returns [2,3,4,1]
  """

    test = np.roll(self.slots,-1* index)
    test1 = Plaintext(test)
    return test1

In [197]:
params = {
        'num_input_channels' : 4, # c_i
        'input_width' : 4, # w_i
        'input_height' : 4, # h_i
        'input_gap' : 2, # k_i
        'kernel_height': 3, # f_h
        'kernel_width': 3, # f_w
        'num_output_channels' : 16, # c_o
        'output_gap' : 4, # k_o
        'output_height' : 2, # h_o
        'output_width' : 2, # w_o
        't_i' : 1, # t_i: c_i/k_i^2, number of squares to represent all input channels
        't_o' : 1 # t_o: c_o/k_o^2, number of squares to represent all input channels
}

input_image = np.array([[[i * params['num_input_channels'] * params['input_width'] + \
                          j * params['num_input_channels'] + k + 1 for k in range(params['num_input_channels'])] \
                         for j in range(params['input_width'])] for i in range(params['input_height'])])
print(input_image) # Should be (input_height, input_width, num_input_channels)

[[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]
  [13 14 15 16]]

 [[17 18 19 20]
  [21 22 23 24]
  [25 26 27 28]
  [29 30 31 32]]

 [[33 34 35 36]
  [37 38 39 40]
  [41 42 43 44]
  [45 46 47 48]]

 [[49 50 51 52]
  [53 54 55 56]
  [57 58 59 60]
  [61 62 63 64]]]


In [198]:
def mult_pack(A, params):
    """
    Args:
        A (numpy.ndarray): Input tensor 
        h_i : Height 
        w_i : Width 
        c_i: Number of input channels 
        params['input_gap']: Input gap
        t_i

    Returns:
        A plaintext object which has a 1-D Numpy array slots.
    """
    # Initialize the output tensor A' with zeros
    A_prime = np.zeros((params['input_gap'] * params['input_height'], params['input_gap'] * params['input_width'], params['t_i']))

    # Fill the A_prime tensor according to the given formula
    for i3 in range(params['input_gap'] * params['input_height']):  # Outer tensor height dimension
        for i4 in range(params['input_gap'] * params['input_width']):  # Outer tensor width dimension
            for i5 in range(params['t_i']):  # Channel dimension after packing
                # Map indices from A to A' based on the input gap
                orig_i3 = i3 // params['input_gap']
                orig_i4 = i4 // params['input_gap']
                orig_i5 = ((params['input_gap'] ** 2) * i5) + params['input_gap'] * (i3 % params['input_gap']) + (i4 % params['input_gap'])

                if (
                    orig_i3 < A.shape[0]
                    and orig_i4 < A.shape[1]
                    and orig_i5 < A.shape[2]
                    and orig_i5 < params['num_input_channels']
                ):
                    A_prime[i3, i4, i5] = A[orig_i3, orig_i4, orig_i5]

    return Plaintext(A_prime.flatten())

In [199]:
def SumSlots(ct_a, m, p):
    """
    SumSlots algorithm implementation using only add, multiply, and rotate.
    Args:
        ct_a (Plaintext): Input ciphertext.
        m (int): Number of added slots
        p (int): Gap
    
    """
    ct_b = [ct_a]

    for j in range(1, int(np.log2(m)) + 1):  # Inclusive loop
        rotated = ct_b[j - 1].rotate(2**(j - 1) * p)
        ct_b.append(ct_b[j - 1].add(rotated))  

    ct_c = ct_b[int(np.log2(m))]

    for j in range(0, int(np.log2(m))):  # Exclusive loop
        if (m // (2**j)) % 2 == 1:
            rotation_distance = (m // (2**(j + 1))) * 2**(j + 1) * p
            rotated = ct_b[j].rotate(rotation_distance)
            ct_c = ct_c.add(rotated)  

    return ct_c #return ciphertext

In [200]:
# ex:
ct_a = Plaintext([1, 2, 3, 4, 5, 6, 7, 8])  # Example slots
m = 4  # Number of slots
p = 2  # Gap

# Compute SumSlots
result = SumSlots(ct_a, m, p)
print("Result:", result)

Result: [16 20 16 20 16 20 16 20]


In [201]:
def U_prime(U, i1, i2, i, params):
    """
    Creates the multiplexed shifted weight tensor U'
    
    Args:
        U: Original  tensor of shape (fh, fw, ci, co)
        i1: Index for filter height dimension
        i2: Index for filter width dimension
        i: Index for output channel dimension
        params: Dictionary containing params ki, hi, wi, ci
    
    Returns:
        Tensor U'(i1,i2,i) of shape (ki*hi, ki*wi, ti)
    """
    
    # Initialize the output tensor with zeros
    result = np.zeros((params['input_gap'] * params['input_height'], params['input_gap'] * params['input_width']), dtype=object)
    
    # Fill the tensor according to the conditions
    for i3 in range(params['input_gap'] * params['input_height']):
        for i4 in range(params['input_gap'] * params['input_width']):
            for i5 in range(params['t_i']):
            # Check all conditions (with i5=0)
                cond1 = ((params['input_gap'] ** 2) * i5) + params['input_gap'] * (i3 % params['input_gap']) + \
                            i4 % params['input_gap'] >= params['num_input_channels']
                cond2 = (i3 // params['input_gap']) - (params['kernel_height'] - 1) // 2 + i1 not in range(params['input_height'])
                cond3 = (i4 // params['input_gap']) - (params['kernel_width'] - 1) // 2 + i2 not in range(params['input_width'])
            
                if cond1 or cond2 or cond3:
                    result[i3, i4] = 0
                else:
                # Calculate indices for the original tensor U
                    u_idx = (i1, i2, ((params['input_gap'] ** 2) * i5) + \
                             params['input_gap'] * (i3 % params['input_gap']) + i4 % params['input_gap'], i)
                    result[i3, i4] = U[u_idx]
                
    return result

In [202]:
def Vector_S(prime):
    return prime.flatten()

In [203]:
def S_prime(params, i):
    S_prime = np.zeros((params['output_gap']*params['output_height'], params['output_gap']*params['output_width'], params['t_o']), dtype=int)

    for i3 in range(params['output_gap']*params['output_height']):
        for i4 in range(params['output_gap']*params['output_width']):
            for i5 in range(params['t_o']):
                if ((params['output_gap']**2)*i5) + params['output_gap'] * (i3 % params['output_gap']) + i4 % params['output_gap'] ==i:
                    S_prime[i3, i4, i5] = 1
                else:
                    S_prime[i3, i4, i5] = 0
    return(Plaintext(Vector_S(S_prime)))

In [204]:
i = 0
selecting_tensor = S_prime(params, i)
print(selecting_tensor)

[1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [205]:
def weightU(params):
    weightU = np.array([[[[i * params['num_input_channels'] * params['kernel_width'] * params['num_output_channels'] + \
                              j * params['num_input_channels'] * params['num_output_channels'] + k * params['num_output_channels'] + \
                              l + 1 
                              for l in range(params['num_output_channels'])] \
                              for k in range(params['num_input_channels'])] \
                              for j in range(params['kernel_width'])] \
                              for i in range(params['kernel_height'])])
    '''
    weightU = np.zeros((params['kernel_height'], params['kernel_width'], params['num_input_channels'], params['num_output_channels']))
    count = 1
    for i in range(params['kernel_height']):
        for j in range(params['kernel_width']):
            for k in range(params['num_input_channels']):
                for l in range(params['num_output_channels']):
                    weightU[i][j][k][l] = count
                    count += 1
    print(weightU)
    '''
    return weightU

In [206]:
def mult_wgt(U, i1, i2, i, params):
    """
    Implements the simplified MultWgt function with ti=1 and i5=0.
    
    Args:
        U: Weight tensor (params['kernel_height'], fw, ci, co)
        i1: Index for filter height 
        i2: Index for filter width 
        i: Index for output channel 
        params: Dictionary containing params (ki, hi, wi, ci)
    
    Returns:
        Vector of U'(multiplexed shifted weight tensor)
    """
    
    # Create the multiplexed shifted weight tensor
    multiplexed = U_prime(U, i1, i2, i, params)
    
    # flatten the tensor 
    return multiplexed.flatten()

In [207]:
def multConv(ct_aprime, U, params):
    num_slots = len(ct_aprime.slots)
    zero_slots = np.zeros(num_slots)
    ct_zero = Plaintext(zero_slots)
    ct_d = ct_zero
    ct_prime = {}
    for i1 in range(params['kernel_height']):
        for i2 in range(params['kernel_width']):
            rotation = params['output_gap'] ** 2 * ((i1 - (params['kernel_height'] - 1) // 2) // 2) \
                        + params['output_gap'] * ((i2 - (params['kernel_width'] - 1) // 2 ) // 2)
            ct_prime[(i1, i2)] = ct_aprime.rotate(rotation)
    for i in range(params['num_output_channels']):
        ct_b = ct_zero
        for i1 in range(params['kernel_height']):
            for i2 in range(params['kernel_width']):
                weight = mult_wgt(U, i1, i2, i, params).flatten()
                #print("(i1, i2, i): " + str(i1) + ", " + str(i2) + ", " + str(i))
                #print(weight)
                weight_p = Plaintext(weight)
                ct_b = ct_b.add(ct_prime[(i1, i2)].mul(weight_p))
        ct_c = SumSlots(ct_b, params['output_gap'], 1)
        ct_c = SumSlots(ct_c, params['output_gap'], params['output_gap'] * params['output_width'])
        ct_c = SumSlots(ct_c, 1, (params['output_gap'] ** 2) * params['output_height'] * params['output_width'])
        rot = -(((i // (params['output_gap'] ** 2)) * (params['output_gap'] ** 2) * params['output_height'] * params['output_width']) + \
                (((i % (params['output_gap']**2)) // params['output_gap']) * params['output_gap'] * params['output_width']) + \
                (i % params['output_gap']))
        ct_d = ct_d.add(ct_c.rotate(rot).mul(S_prime(params, i)))
    return ct_d

In [208]:
ct_aprime = mult_pack(input_image, params)
U = weightU(params)
print(U.shape)
ct_d = multConv(ct_aprime, U, params)
print(ct_d)

(3, 3, 4, 16)
[490282.0 491796.0 493310.0 494824.0 521274.0 522804.0 524334.0 525864.0
 496338.0 497852.0 499366.0 500880.0 527394.0 528924.0 530454.0 531984.0
 502394.0 503908.0 505422.0 506936.0 533514.0 535044.0 536574.0 538104.0
 508450.0 509964.0 511478.0 512992.0 539634.0 541164.0 542694.0 544224.0
 1004714.0 1008404.0 1012094.0 1015784.0 1016634.0 1020724.0 1024814.0
 1028904.0 1019474.0 1023164.0 1026854.0 1030544.0 1032994.0 1037084.0
 1041174.0 1045264.0 1034234.0 1037924.0 1041614.0 1045304.0 1049354.0
 1053444.0 1057534.0 1061624.0 1048994.0 1052684.0 1056374.0 1060064.0
 1065714.0 1069804.0 1073894.0 1077984.0]


In [209]:
def zeroBorder(array):
  """Method adds a border of zeros around the array
  Args:
    n: length of the array
  Returns:
    borderArray: array with a border of zeros
  """
  if array.ndim == 2:
      n = array.shape[0]
      borderArray = np.zeros((n+2, n+2))
      borderArray[1:-1, 1:-1] = array
  elif array.ndim == 3:
      c, n, m = array.shape
      borderArray = np.zeros((c, n+2, m+2))
      borderArray[:, 1:-1, 1:-1] = array
  return borderArray

In [210]:
def outputDimensions(image, stride, kernel_height, kernel_width):
    borderArray = zeroBorder(image)
    _, padded_height, padded_width = borderArray.shape 
    outputHeight = ((padded_height - kernel_height) // stride) + 1
    outputWidth = ((padded_width - kernel_width) // stride) + 1
    return borderArray, outputHeight, outputWidth

In [211]:
def twod_naive_convolution(image, kernel, borderArray, image_height, image_width, kernel_height, kernel_width, stride, outputHeight, outputWidth):
  """Method computes the convolution of an image and a kernel with only one output channel
  Args:
    image: image as a 1D array
    kernel: kernel as a 1D array
    borderArray: image with a border of zeros
    outputImage: convoluted image
  Returns:
    The convoluted image
  """
  
  image = image.reshape(image_height, image_width)
  kernel = kernel.reshape(kernel_height, kernel_width)

  #borderArray = zeroBorder(image)
  #padded_height, padded_width = borderArray.shape 

  #print(f"The padded image is \n {borderArray}")
  #outputHeight = ((padded_height - kernel_height) // stride) + 1
  #outputWidth = ((padded_width - kernel_width) // stride) + 1
  output_Image = np.zeros((outputHeight, outputWidth))
    
  for i in range(outputHeight):
      for j in range(outputWidth):
          start_i = i * stride
          start_j = j * stride
          matrix = borderArray[start_i : start_i + kernel_height, start_j : start_j + kernel_width]
          mult = np.multiply(matrix, kernel)
          add = np.sum(mult)
          output_Image[i, j] = add
  return output_Image

In [212]:
def threed_naive_convolution(image, kernel, image_height, image_width, num_input_channels, num_output_channels, kernel_height, kernel_width, stride):
    borderArray, output_Height, output_Width = outputDimensions(image, stride, kernel_height, kernel_width) 
    print(output_Height, output_Width, num_output_channels)
    output = np.zeros((output_Height, output_Width, num_output_channels))

    #print(output.shape)
    for i in range(num_output_channels):
      #print(f"Output Channel {i}:\n{output[:, :, i]}")
      for j in range(num_input_channels):
        #print(f"Kernel for input channel {j} -> output channel {i}:\n{kernel[j, i]}")
        convolution = twod_naive_convolution(image[j], kernel[:, :, j,i], borderArray[j], image_height, image_width, kernel_height, kernel_width, stride, output_Height, output_Width)
        #print(f"Convolution result for input channel {j}, output channel {i}:\n{convolution}")
        output[:,:,i] += convolution
        #print(f"Output channel {i} after input channel {j} contribution:\n{output[:, :, i]}")

    print(output.shape)
    return output

In [213]:
#input1 = np.array([[1,2,3],[4,5,6],[7,8,9]])
#input2 = np.array([[2,3,4],[5,6,7],[8,9,10]])
#inputchannels = np.array([input1, input2])
#threedkernel1 = np.array([[1,2,3],[4,5,6],[7,8,9]])
#threedkernel2 = np.array([[2,3,4],[5,6,7],[8,9,10]])
#threedkernel3 = np.array([[1,2,3],[4,5,6],[7,8,9]])
#threedkernel4 = np.array([[2,3,4],[5,6,7],[8,9,10]])
#threedkernel5 = np.array([[1,2,3],[4,5,6],[7,8,9]])
#threedkernel6 = np.array([[2,3,4],[5,6,7],[8,9,10]])
#threedkernel7 = np.array([[1,2,3],[4,5,6],[7,8,9]])
#threedkernel8 = np.array([[2,3,4],[5,6,7],[8,9,10]])
#threedkernel = np.array([threedkernel1, threedkernel2, threedkernel3, threedkernel4, threedkernel5, threedkernel6, threedkernel7, threedkernel8])
#num_output_channels = int(threedkernel.shape[0]/inputchannels.shape[0])
#print(threedkernel.shape)
#print(threedkernel.size)
#threedkernel = threedkernel.reshape(inputchannels.shape[0], num_output_channels, 3, 3)
#convoluted_Image = threed_naive_convolution(inputchannels, threedkernel, inputchannels.shape[1], inputchannels.shape[2], inputchannels.shape[0], num_output_channels, threedkernel.shape[2], threedkernel.shape[3], 1)

#print(convoluted_Image)

In [214]:
# Check that the above output matches multPack(C)
# C = 3d-conv(input_image, U, params) with stride 2
# Implement 3d-conv with stride parameter and test with s = 2

In [215]:
U = U.reshape(U.shape[0], U.shape[1], params['num_input_channels'], params['num_output_channels'] )
C = threed_naive_convolution(input_image, U, input_image.shape[1], input_image.shape[2], input_image.shape[0], num_output_channels, U.shape[0], U.shape[1], 2)
mult_pack(C, params)

2 2 4
(2, 2, 4)
